In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! unzip /content/drive/MyDrive/Colab/2212_MLSP/mlsp_contest_dataset.zip 

串流輸出內容已截斷至最後 5000 行。
  inflating: mlsp_contest_dataset/essential_data/src_wavs/PC13_20090513_070000_0010.wav  
  inflating: __MACOSX/mlsp_contest_dataset/essential_data/src_wavs/._PC13_20090513_070000_0010.wav  
  inflating: mlsp_contest_dataset/essential_data/src_wavs/PC13_20090513_070000_0020.wav  
  inflating: __MACOSX/mlsp_contest_dataset/essential_data/src_wavs/._PC13_20090513_070000_0020.wav  
  inflating: mlsp_contest_dataset/essential_data/src_wavs/PC13_20090513_070000_0030.wav  
  inflating: __MACOSX/mlsp_contest_dataset/essential_data/src_wavs/._PC13_20090513_070000_0030.wav  
  inflating: mlsp_contest_dataset/essential_data/src_wavs/PC13_20090513_070000_0040.wav  
  inflating: __MACOSX/mlsp_contest_dataset/essential_data/src_wavs/._PC13_20090513_070000_0040.wav  
  inflating: mlsp_contest_dataset/essential_data/src_wavs/PC13_20090606_050000_0010.wav  
  inflating: __MACOSX/mlsp_contest_dataset/essential_data/src_wavs/._PC13_20090606_050000_0010.wav  
  inflating: mlsp_contes

In [ ]:
import pandas as pd
import numpy as np
import librosa
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchsummary import summary
import torchaudio
from PIL import Image
from PIL import ImageFilter

In [ ]:
#Dataset

cvfold = pd.read_csv("/content/mlsp_contest_dataset/essential_data/CVfolds_2.txt")
id2file = pd.read_csv("/content/mlsp_contest_dataset/essential_data/rec_id2filename.txt")
sample_label = pd.read_csv("/content/mlsp_contest_dataset/essential_data/rec_labels_test_hidden.txt", header=None, sep='\n')
sample_label = sample_label[0].str.split(",", expand=True)
sample_label.drop([0], inplace=True)
sample_label.reset_index(drop=True, inplace=True)
sample_label.head()

,0,1,2,3,4,5,6
0,0,?,None,None,None,None,None
1,1,?,None,None,None,None,None
2,2,11,12,None,None,None,None
3,3,None,None,None,None,None,None
4,4,?,None,None,None,None,None


In [ ]:
train_recid = []
test_recid = []
for i in range(len(cvfold['rec_id'])):
  if cvfold['fold'][i] == 0:
    train_recid.append(cvfold['rec_id'][i])
  elif cvfold['fold'][i] == 1:
    test_recid.append(cvfold['rec_id'][i])
  else:
    print('CVfold tag error.')

print('Num of train data: ', len(train_recid))
print('Num of test data: ', len(test_recid))

Num of train data:  322
Num of test data:  323


In [ ]:
train_img_path = []
test_img_path = []
for i in range(len(id2file['rec_id'])):
  for j in range(len(train_recid)):
    if id2file['rec_id'][i] == train_recid[j]:
      train_img_path.append('/content/mlsp_contest_dataset/supplemental_data/filtered_spectrograms/' + id2file['filename'][i] + '.bmp')
  for k in range(len(test_recid)):
    if id2file['rec_id'][i] == test_recid[k]:
      test_img_path.append('/content/mlsp_contest_dataset/supplemental_data/filtered_spectrograms/' + id2file['filename'][i] + '.bmp')
train_img_path = pd.DataFrame(train_img_path, columns=['img_path'])
test_img_path = pd.DataFrame(test_img_path, columns=['img_path'])
print('Num of train path: ', len(train_img_path))
print('Num of test path: ', len(test_img_path))

Num of train path:  322
Num of test path:  323


In [ ]:
train_label = sample_label.drop(test_recid)
train_label.reset_index(drop=True, inplace=True)

y_label = pd.DataFrame(columns=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18'],
                       index = list(range(len(train_recid))))
y_label.fillna(0, inplace = True)
for i in range(len(train_label[0])):
  for j in range(1,7):
    for k in range(19):
      if train_label[j][i] != None:
        if int(train_label[j][i]) == k:
          y_label.iat[i, k] = 1

y_label

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
318,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
319,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
320,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0


In [ ]:
from torch.utils.data.dataset import Dataset
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
import os

class MultiLabelDataset(Dataset):
    def __init__(self, transform=None):
        
        self.transform = transform
        self.X_train = train_img_path['img_path']
        self.y_train = y_label.to_numpy()

    def __getitem__(self, index):
        img = Image.open(self.X_train[index])
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        return len(self.X_train.index)

In [ ]:
def binary_cross_entropy_with_logits(input, target):
    if not target.is_same_size(input):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), input.size()))
    max_val = (-input).clamp(min=0)
    loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()

    return loss.mean()

In [ ]:
import torch.optim as optim
from torch.autograd import Variable
import torchvision.datasets as dset
from torch.utils.data import DataLoader
import torchvision.models as models
import sys
import math

class Bottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(Bottleneck, self).__init__()
        interChannels = 4*growthRate
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, interChannels, kernel_size=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

class SingleLayer(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(SingleLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = torch.cat((x, out), 1)
        return out

class Transition(nn.Module):
    def __init__(self, nChannels, nOutChannels):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, nOutChannels, kernel_size=1,
                               bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNetBase(nn.Module):
    def __init__(self, growthRate, depth, reduction, nClasses, bottleneck):
        super(DenseNetBase, self).__init__()

        nDenseBlocks = (depth-4) // 3
        if bottleneck:
            nDenseBlocks //= 2

        nChannels = 2*growthRate
        self.conv1 = nn.Conv2d(3, nChannels, kernel_size=3, padding=1,
                               bias=False)
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans1 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans2 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense3 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate

        self.bn1 = nn.BatchNorm2d(nChannels)
        self.fc = nn.Linear(nChannels, nClasses)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(Bottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.dense3(out)
        out = torch.squeeze(F.avg_pool2d(F.relu(self.bn1(out)), 8))
        # out = out.view([out.size()[0], -1])
        return self.fc(out)

class DenseNet121(nn.Module):
    def __init__(self, num_classes, pretrained):
        super(DenseNet121, self).__init__()
        net = models.densenet.densenet121(pretrained=pretrained)
        self.features = net.features
        self.classifier = nn.Linear(1024, num_classes)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.avg_pool2d(out, kernel_size=7).view(features.size(0), -1)
        return self.classifier(out)

class DenseNet100(models.densenet.DenseNet):
    def __init__(self, num_classes):
        super(DenseNet100, self).__init__(growth_rate=12, block_config=(16, 16, 16), num_init_features=24, bn_size=3, drop_rate=0.5, num_classes=num_classes)
    
    def forward(self, x):
        out = torch.squeeze(F.avg_pool2d(F.relu(self.features(x)), 8))
        return self.classifier(out)

In [ ]:
import os
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.models as models



def get_loss_function(multilabel):
    if multilabel:
        return binary_cross_entropy_with_logits
    return F.cross_entropy

def get_dataset(name, partition, transform):
    split = 0.8
    all_datasets = MultiLabelDataset(transform=transform)
    train_size = int(split * len(train_img_path))
    test_size = len(train_img_path) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(all_datasets,[train_size, test_size])
    if partition == 'train':
       return train_dataset
    elif partition == 'test':
       return test_dataset

def get_network(name, num_classes, pretrained):
    if name == 'densenet-base':
        assert not pretrained
        return DenseNetBase(growthRate=12, depth=100, reduction=0.5, bottleneck=True, nClasses=num_classes)
    elif name == 'densenet-121':
        return DenseNet121(num_classes, pretrained) 
    elif name == 'densenet-100':
        assert not pretrained
        return DenseNet100(num_classes)

In [ ]:
pip install setproctitle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Train
import argparse
import setproctitle
import shutil

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('-n', '--network-name', type=str, required=False, default='densenet-121')
    parser.add_argument('-d', '--dataset-name', type=str, required=False, default='MLSP')
    parser.add_argument('-c', '--num-classes', type=int, required=False, default=19)
    parser.add_argument('-m', '--multilabel', type=bool, default=True)
    parser.add_argument('-p', '--pretrained', type=bool, default=True)
    parser.add_argument('-l', '--load')
    parser.add_argument('--batchSz', type=int, default=64)
    parser.add_argument('--nEpochs', type=int, default=300)
    parser.add_argument('--sEpoch', type=int, default=1)
    parser.add_argument('--no-cuda', action='store_true')
    parser.add_argument('--save')
    parser.add_argument('--seed', type=int, default=50)
    parser.add_argument('--opt', type=str, default='sgd', choices=('sgd', 'adam', 'rmsprop'))
    args = parser.parse_args(args=[])
    args.network_name = 'densenet-121'
    args.dataset_name = 'MLSP'
    args.num_classes = 19

    args.cuda = not args.no_cuda and torch.cuda.is_available()
    args.save = args.save or '/content/drive/MyDrive/Colab/2212_MLSP/work/%s/%s' % (args.network_name, args.dataset_name)
    setproctitle.setproctitle(args.save)

    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)

    if os.path.exists(args.save):
        shutil.rmtree(args.save)
    os.makedirs(args.save, exist_ok=True)

    normMean = [0.49139968, 0.48215827, 0.44653124]
    normStd = [0.24703233, 0.24348505, 0.26158768]
    normTransform = transforms.Normalize(normMean, normStd)

    trainTransform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop(224),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
	      #RandomBrightness(-0.25, 0.25),
        normTransform
    ])
    testTransform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normTransform
    ])

    kwargs = {'num_workers': 6, 'pin_memory': True} if args.cuda else {}
    trainLoader = DataLoader(
        get_dataset(args.dataset_name, 'train', trainTransform),
        batch_size=args.batchSz, shuffle=True, **kwargs)
    testLoader = DataLoader(
        get_dataset(args.dataset_name, 'test', testTransform),
        batch_size=args.batchSz, shuffle=False, **kwargs)

    if args.load:
        print("Loading network: {}".format(args.load))
        net = torch.load(args.load)
    else:
        net = get_network(args.network_name, args.num_classes, args.pretrained)
    
    if True: # make this an optional
        net = torch.nn.DataParallel(net)
    
    print('  + Number of params: {}'.format(sum([p.data.nelement() for p in net.parameters()])))
    if args.cuda:
        net = net.cuda() #.half()

    if args.opt == 'sgd':
        optimizer = optim.SGD(net.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
    elif args.opt == 'adam':
        optimizer = optim.Adam(net.parameters(), weight_decay=1e-4)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(net.parameters(), weight_decay=1e-4)

    trainF = open(os.path.join(args.save, 'train.csv'), 'a')
    testF = open(os.path.join(args.save, 'test.csv'), 'a')

    for epoch in range(args.sEpoch, args.nEpochs + args.sEpoch):
        adjust_opt(args.opt, optimizer, epoch)
        train(args, epoch, net, trainLoader, optimizer, trainF)
        test(args, epoch, net, testLoader, optimizer, testF)
        torch.save(net, os.path.join(args.save, '%d.pth' % epoch))

    trainF.close()
    testF.close()

def train(args, epoch, net, trainLoader, optimizer, trainF):
    net.train()
    nProcessed = 0
    nTrain = len(trainLoader.dataset)
    for batch_idx, (data, target) in enumerate(trainLoader):
        if args.cuda:
            data = data.cuda() #.half()
            if args.multilabel:
                target = target.cuda() #.half()
            else:
                target = target.cuda() #.long()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = net(data)
        loss = get_loss_function(args.multilabel)(output, target)
        loss.backward()
        optimizer.step()
        nProcessed += len(data)
        if args.multilabel:
            pred = output.data.gt(0.5)
            tp = (pred + target.data.byte()).eq(2).sum()
            fp = (pred.int() - target.data.byte()).eq(1).sum()
            fn = (pred.int() - target.data.byte()).eq(-1).sum()
            tn = (pred + target.data.byte()).eq(0).sum()
            acc = (tp + tn) / (tp + tn + fp + fn)
            try:
                prec = tp / (tp + fp)
            except ZeroDivisionError:
                prec = 0.0
            try:
                rec = tp / (tp + fn)
            except ZeroDivisionError:
                rec = 0.0
            partialEpoch = epoch + batch_idx / len(trainLoader) - 1
            print('Epoch: {:.2f} [{}/{} ({:.0f}%)]\tLoss: {:.4f}\tAcc: {:.4f}\tPrec: {:.4f}\tRec: {:.4f}\tTP: {}\tFP: {}\tFN: {}\tTN: {}'.format(
                partialEpoch, nProcessed, nTrain, 100. * batch_idx / len(trainLoader),
                loss.data, acc, prec, rec, tp, fp, fn, tn))
            trainF.write('{},{},{},{},{}\n'.format(partialEpoch, loss.data, acc, prec, rec))
        else:
            pred = output.data.max(1)[1]
            incorrect = pred.ne(target.data).sum()
            err = 100.*incorrect/len(data)
            partialEpoch = epoch + batch_idx / len(trainLoader) - 1
            print('Train Epoch: {:.2f} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tError: {:.6f}'.format(
                partialEpoch, nProcessed, nTrain, 100. * batch_idx / len(trainLoader),
                loss.data, err))
            trainF.write('{},{},{}\n'.format(partialEpoch, loss.data, err))
        trainF.flush()

def test(args, epoch, net, testLoader, optimizer, testF):
    net.eval()
    test_loss = 0
    acc = prec = rec = 0
    incorrect = 0
    for data, target in testLoader:
        if args.cuda:
            data = data.cuda() #.half()
            if args.multilabel:
                target = target.cuda() #.half()
            else:
                target = target.cuda() #.long()
        data, target = Variable(data, volatile=True), Variable(target)
        output = net(data)
        test_loss += get_loss_function(args.multilabel)(output, target).data
        if args.multilabel:
            pred = output.data.gt(0.5)
            tp = (pred + target.data.byte()).eq(2).sum()
            fp = (pred.int() - target.data.byte()).eq(1).sum()
            fn = (pred.int() - target.data.byte()).eq(-1).sum()
            tn = (pred + target.data.byte()).eq(0).sum()
            acc += (tp + tn) / (tp + tn + fp + fn)
            try:
                prec += tp / (tp + fp)
            except ZeroDivisionError:
                prec += 0.0
            try:
                rec += tp / (tp + fn)
            except ZeroDivisionError:
                rec += 0.0
        else:
            pred = output.data.max(1)[1] # get the index of the max log-probability
            incorrect += pred.ne(target.data).sum()
    test_loss /= len(testLoader)
    acc /= len(testLoader)
    prec /= len(testLoader)
    rec /= len(testLoader)
    if args.multilabel:
        print('\nTest set: Loss: {:.4f}, Acc: {:.4f}, Prec: {:.4f}, Rec: {:.4f}\n'.format(
            test_loss, acc, prec, rec))
        testF.write('{},{},{},{},{}\n'.format(epoch, test_loss, acc, prec, rec))
    else:
        nTotal = len(testLoader.dataset)
        err = 100. * incorrect / nTotal
        print('\nTest set: Average loss: {:.4f}, Error: {}/{} ({:.0f}%)\n'.format(
        test_loss, incorrect, nTotal, err))
        testF.write('{},{},{}\n'.format(epoch, test_loss, err))
    testF.flush()

def adjust_opt(optAlg, optimizer, epoch):
    if optAlg == 'sgd':
        if epoch < 150: lr = 1e-1
        elif epoch == 150: lr = 1e-2
        elif epoch == 225: lr = 1e-3
        else: return

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

if __name__=='__main__':
    main()

In [ ]:
#Show results
import argparse
import csv
import os

parser = argparse.ArgumentParser()
parser.add_argument('-n', '--network-name', type=str, required=False)
parser.add_argument('-d', '--dataset-name', type=str, required=False)
parser.add_argument('-m', '--multilabel', type=bool, default=True)
args = parser.parse_args(args=[])
args.network_name = 'densenet-121'
args.dataset_name = 'MLSP'

results_dir = os.path.join('/content/drive/MyDrive/Colab/2212_MLSP/work', args.network_name, args.dataset_name)
train_file = os.path.join(results_dir, 'train.csv')
test_file = os.path.join(results_dir, 'test.csv')

with open(train_file, 'r') as csvfile:
        i = rows = loss = prec = acc = rec = err = 0
        reader = csv.reader(csvfile)
        
        print('TRAIN\n')
        if args.multilabel:
            print('Epoch\tLoss\t\tAccuracy\tPrecision\tRecall')
            for row in reader:
                if int(float(row[0])) == i:
                    loss += float(row[1])
                    acc += float(row[2])
                    prec += float(row[3])
                    rec += float(row[4])
                    rows += 1
                else:
                    print('%d\t%.6f\t%.6f\t%.6f\t%.6f' % (i + 1, loss / rows, acc / rows, prec / rows, rec / rows))
                    i += 1
                    loss = float(row[1])
                    acc = float(row[2])
                    prec = float(row[3])
                    rec = float(row[4])
                    rows = 1
            print('%d\t%.6f\t%.6f\t%.6f\t%.6f' % (i + 1, loss / rows, acc / rows, prec / rows, rec / rows))
        else:
            print('Epoch\tLoss\t\tError')
            for row in reader:
                if int(float(row[0])) == i:
                    loss += float(row[1])
                    err += float(row[2])
                    rows += 1
                else:
                    print('%d\t%.6f\t%.6f' % (i + 1, loss / rows, err / rows))
                    i += 1
                    loss = float(row[1])
                    err = float(row[2])
                    rows = 1
            print('%d\t%.6f\t%.6f' % (i + 1, loss / rows, err / rows))

with open(test_file, 'r') as csvfile:
    reader = csv.reader(csvfile)

    print('\nTEST\n')
    if args.multilabel:
        print('Epoch\tLoss\t\tAccuracy\tPrecision\tRecall')
        for row in reader:
            epoch = int(row[0])
            loss = float(row[1])
            acc = float(row[2])
            prec = float(row[3])
            rec = float(row[4])
            print('%d\t%.6f\t%.6f\t%.6f\t%.6f' % (epoch, loss, acc, prec, rec))
    else:
        print('Epoch\tLoss\t\tError')
        for row in reader:
            epoch = int(row[0])
            loss = float(row[1])
            err = float(row[2])
            print('%d\t%.6f\t%.6f' % (epoch, loss, err))

TRAIN

Epoch	Loss		Accuracy	Precision	Recall
1	0.367725	0.915132	nan	0.030000
2	0.322705	0.919737	nan	0.074306
3	0.182196	0.952303	nan	nan
4	0.420361	0.921875	nan	0.000000
5	0.235059	0.928289	nan	0.005520
6	0.225928	0.918914	nan	nan
7	0.137651	0.954112	nan	nan
8	0.140282	0.952961	nan	nan
9	0.129289	0.953289	nan	nan
10	0.181726	0.944737	nan	0.045269
11	0.444751	0.932401	nan	0.083949
12	0.353516	0.939145	nan	0.172208
13	0.135927	0.959375	nan	nan
14	0.249658	0.950000	nan	0.143258
15	0.118587	0.962500	nan	nan
16	0.241870	0.940296	nan	0.239699
17	0.129285	0.950493	0.542339	nan
18	0.107434	0.964967	nan	nan
19	0.093336	0.964145	nan	nan
20	0.085035	0.968257	nan	nan
21	0.077938	0.970888	nan	nan
22	0.071147	0.974013	nan	nan
23	0.065959	0.974671	nan	nan
24	0.061288	0.975164	nan	nan
25	0.424432	0.935526	nan	0.428386
26	0.172095	0.969243	nan	0.490438
27	0.298535	0.938980	nan	0.187326
28	0.186316	0.955428	nan	nan
29	0.317041	0.919901	nan	0.217615
30	0.159827	0.953947	nan	0.226092
31	0.138965	0.96825

In [ ]:
#Predict
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_network('densenet-121', 19, True)
model = torch.load('/content/drive/MyDrive/Colab/2212_MLSP/work/densenet-121/MLSP/300.pth')
model = model.to(device)
model.eval()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DataParallel(
  (module): DenseNet121(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momen

In [ ]:
normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]
normTransform = transforms.Normalize(normMean, normStd)
testTransform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normTransform
    ])

predictions = []
for i in range(len(test_img_path)):
  img = Image.open(test_img_path['img_path'][i])
  img = img.convert('RGB')
  img = testTransform(img)
  img = img.cuda()
  img = img.unsqueeze(0)

  output = model(img)
  prob = torch.sigmoid(output)
  prob1 = prob.cpu().detach().numpy()

  predictions.append(prob1[0])

In [ ]:
output_prediction = pd.DataFrame(predictions, columns = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18'])
output_prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.000098,0.001147,0.000904,0.000021,0.012192,0.000061,0.001912,0.026764,0.004250,0.003780,0.000575,0.999512,1.000000,0.000037,0.000883,0.000082,0.000247,0.000492,0.020889
1,0.005119,0.004627,0.003824,0.001803,0.005241,0.004738,0.008186,0.007038,0.006290,0.006145,0.006145,0.005363,0.004520,0.001810,0.003870,0.003708,0.008743,0.002823,0.006340
2,0.000006,0.002823,0.000274,0.000009,0.000167,0.000098,0.001399,0.045776,0.001073,0.000169,1.000000,0.000120,0.001040,0.000019,0.000272,0.000631,0.000171,0.000016,0.000947
3,0.004505,0.004887,0.003839,0.002350,0.006168,0.005260,0.009270,0.009056,0.006641,0.007206,0.007122,0.005344,0.007011,0.002140,0.005512,0.004330,0.008034,0.003111,0.007755
4,0.006313,0.004025,0.003975,0.001305,0.004200,0.004700,0.007404,0.006363,0.005890,0.005600,0.004299,0.003901,0.003063,0.001315,0.003456,0.003075,0.005642,0.002296,0.004848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0.001235,0.001919,0.001472,0.000458,0.001889,0.001389,0.004005,0.002512,0.003147,0.002779,0.003525,0.001694,0.001675,0.000477,0.001174,0.001335,0.002663,0.001004,0.002714
318,0.988770,0.000762,0.002981,0.000148,0.000448,0.080383,0.002157,0.011871,0.003111,0.002878,0.000092,0.002132,0.000289,0.000355,0.001221,0.000234,0.001202,0.000405,0.000669
319,0.001775,0.000863,0.003052,0.000043,0.005280,0.000346,0.004005,0.019165,1.000000,0.007290,0.004025,0.001897,0.002398,0.000025,0.004810,0.001362,0.000150,0.000234,0.000341
320,0.000031,0.000222,0.000155,0.000005,0.994141,0.000187,0.000055,0.000062,1.000000,0.000546,0.999512,0.000688,0.008614,0.000062,1.000000,0.002913,0.000017,0.000007,0.000039


In [ ]:
output_prediction.to_csv('/content/drive/MyDrive/Colab/2212_MLSP/densenet_train.csv', index = False)
#output_prediction.to_csv('/content/drive/MyDrive/Colab/2212_MLSP/densenet_pred.csv', index = False)

In [ ]:
#K-Means clustering of segments
from sklearn import cluster
from scipy.cluster.vq import whiten 

k = 50 
kextra = 10 
nrecs = 645
rseed = 0

segfile = open('/content/mlsp_contest_dataset/supplemental_data/segment_features.txt',
               'r')
line = segfile.readline()
line = segfile.readline()
index = 0
while line != '':
    tokens = line.split(',')
    nums = list(map(float, tokens))
    nums = nums[2:len(line)] # Omit recid and segid

    if index == 0:
        segfeatures = nums
    else:
        segfeatures = np.vstack((segfeatures, nums))

    line = segfile.readline()
    index += 1

segfeatures = whiten(segfeatures)

kmeans1 = cluster.KMeans(n_clusters=k, init='k-means++', n_init=50,
                         max_iter=300, random_state=rseed)
kmeans2 = cluster.KMeans(n_clusters=kextra, init='k-means++', n_init=50,
                         max_iter=300, random_state=rseed)

clusters1 = kmeans1.fit_predict(segfeatures)
clusters2 = kmeans2.fit_predict(segfeatures)

segfile.seek(0)
line = segfile.readline()
line = segfile.readline()
index = 0
prevrecid = -1
hist = np.zeros((nrecs, k + kextra))
while line != '':
    while 1:
        tokens = line.split(',')
        recid = int(tokens[0]) 
        if recid != prevrecid:
            prevrecid = recid
            break

        hist[recid][clusters1[index]] += 1
        hist[recid][k + clusters2[index]] += 1

        line = segfile.readline()
        if line == '':
            break

        index += 1

segfile.close()

histfilename = 'myhist.txt'
histfile = open(histfilename, 'w')
histfile.write('recid,[hist]\n')

for recid in range(nrecs):
    histfile.write('%d,' % recid)   
    for col in range(k + kextra - 1):
        histfile.write('%f,' % hist[recid][col])   

    histfile.write('%f\n' % hist[recid][col + 1])   
histfile.close()

In [ ]:
#Recording location
location = id2file['filename'].str.split("_", expand = True)
location.drop(columns=[1,2,3], inplace=True)
train_loc = []
test_loc = []
for i in range(len(train_recid)):
  train_loc.append(location[0][int(train_recid[i])])
for i in range(len(test_recid)):
  test_loc.append(location[0][int(test_recid[i])])

In [ ]:
loc_map = {'PC1': 0.055, 'PC2': 0.11, 'PC3': 0.165, 'PC4': 0.22, 'PC5': 0.275,
           'PC6': 0.33, 'PC7': 0.385, 'PC8': 0.44, 'PC9': 0.495, 'PC10': 0.55,
           'PC11': 0.605, 'PC12': 0.66, 'PC13': 0.715, 'PC14': 0.77, 'PC15': 0.825,
           'PC16': 0.88, 'PC17': 0.935, 'PC18': 0.99}
train_loc_map = []
test_loc_map = []
for i in range(len(train_loc)):
  train_loc_map.append(loc_map[train_loc[i]]) 
for i in range(len(test_loc)):
  test_loc_map.append(loc_map[test_loc[i]]) 

In [ ]:
#Recording area
train_area_map = []
test_area_map = []
for i in range(len(train_loc_map)):
  if 0 < train_loc_map[i] < 0.35:
    train_area_map.append(0.166)
  elif 0.35 < train_loc_map[i] < 0.45:
    train_area_map.append(0.332)
  elif 0.45 < train_loc_map[i] < 0.5:
    train_area_map.append(0.498)
  elif 0.5 < train_loc_map[i] < 0.75:
    train_area_map.append(0.664)
  elif 0.75 < train_loc_map[i] < 0.9:
    train_area_map.append(0.83)
  elif 0.9 < train_loc_map[i] < 1:
    train_area_map.append(0.996)
for i in range(len(test_loc_map)):
  if 0 < test_loc_map[i] < 0.35:
    test_area_map.append(0.166)
  elif 0.35 < test_loc_map[i] < 0.45:
    test_area_map.append(0.332)
  elif 0.45 < test_loc_map[i] < 0.5:
    test_area_map.append(0.498)
  elif 0.5 < test_loc_map[i] < 0.75:
    test_area_map.append(0.664)
  elif 0.75 < test_loc_map[i] < 0.9:
    test_area_map.append(0.83)
  elif 0.9 < test_loc_map[i] < 1:
    test_area_map.append(0.996)

In [ ]:
hist = pd.read_csv("/content/myhist.txt", skiprows=1, header=None)
hist.drop(columns=0, inplace=True)

In [ ]:
train_hist = hist.drop(test_recid, axis=0)
train_hist.reset_index(drop = True, inplace=True)
test_hist = hist.drop(train_recid, axis=0)
test_hist.reset_index(drop = True, inplace=True)

In [ ]:
train_dense = pd.read_csv("/content/drive/MyDrive/Colab/2212_MLSP/densenet_train.csv")
test_dense = pd.read_csv("/content/drive/MyDrive/Colab/2212_MLSP/densenet_pred.csv")

In [ ]:
train_loc_map = pd.DataFrame(train_loc_map, columns=['loc'])
test_loc_map = pd.DataFrame(test_loc_map, columns=['loc'])
train_area_map = pd.DataFrame(train_area_map, columns=['area'])
test_area_map = pd.DataFrame(test_area_map, columns=['area'])    

In [ ]:
#Training and testing dataset
train_df = pd.concat([train_dense, train_hist, train_loc_map, train_area_map], axis = 1, ignore_index= True)
test_df = pd.concat([test_dense, test_hist, test_loc_map, test_area_map], axis = 1, ignore_index= True)
train_df

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,0.000098,0.001147,0.000904,0.000021,0.012190,0.000061,0.001912,0.026760,0.004250,0.003780,...,0.0,0.0,1.0,0.0,4.0,0.0,2.0,0.0,0.055,0.166
1,0.005120,0.004627,0.003824,0.001803,0.005240,0.004738,0.008190,0.007040,0.006290,0.006145,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055,0.166
2,0.000006,0.002823,0.000274,0.000009,0.000167,0.000098,0.001399,0.045780,0.001073,0.000169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055,0.166
3,0.004505,0.004887,0.003840,0.002350,0.006170,0.005260,0.009270,0.009056,0.006640,0.007206,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055,0.166
4,0.006313,0.004025,0.003975,0.001305,0.004200,0.004700,0.007404,0.006363,0.005890,0.005600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055,0.166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0.001235,0.001919,0.001472,0.000458,0.001889,0.001389,0.004005,0.002512,0.003147,0.002779,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.935,0.996
318,0.989000,0.000762,0.002981,0.000148,0.000448,0.080400,0.002157,0.011870,0.003110,0.002878,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.935,0.996
319,0.001775,0.000863,0.003052,0.000043,0.005280,0.000346,0.004005,0.019170,1.000000,0.007290,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.990,0.996
320,0.000031,0.000222,0.000155,0.000005,0.994000,0.000187,0.000055,0.000062,1.000000,0.000547,...,0.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.990,0.996


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
X = train_df
y = y_label

In [ ]:
#parameter optimization
params = {'estimator__n_estimators': range(500,2000,100)}
clf = MultiOutputClassifier(RandomForestClassifier(n_jobs = -1))
gsearch = GridSearchCV(clf, params)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'estimator__n_estimators': 1700}, 0.7421634615384616)

In [ ]:
params = {'estimator__max_features': ['sqrt', 'log2', None]}
clf = MultiOutputClassifier(RandomForestClassifier(n_jobs = -1))
gsearch = GridSearchCV(clf, params)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'estimator__max_features': None}, 0.9006730769230769)

In [ ]:
params = {'estimator__max_features': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}
clf = MultiOutputClassifier(RandomForestClassifier(n_jobs = -1))
gsearch = GridSearchCV(clf, params)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'estimator__max_features': 0.7}, 0.913125)

In [ ]:
params = {'estimator__min_samples_leaf': range(1,50,5)}
clf = MultiOutputClassifier(RandomForestClassifier(n_jobs = -1))
gsearch = GridSearchCV(clf, params)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'estimator__min_samples_leaf': 1}, 0.7327884615384616)

In [ ]:
params = {'estimator__min_samples_split': range(2,50,5)}
clf = MultiOutputClassifier(RandomForestClassifier(n_jobs = -1))
gsearch = GridSearchCV(clf, params)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'estimator__min_samples_split': 7}, 0.7266826923076923)

In [ ]:
params = {'estimator__max_depth': range(2,50,5)}
clf = MultiOutputClassifier(RandomForestClassifier(n_jobs = -1))
gsearch = GridSearchCV(clf, params)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'estimator__max_depth': 22}, 0.7359615384615384)

In [ ]:
params = {'estimator__random_state': [None, np.random.RandomState(0)]}
clf = MultiOutputClassifier(RandomForestClassifier(n_jobs = -1))
gsearch = GridSearchCV(clf, params)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'estimator__random_state': None}, 0.7265865384615384)

In [ ]:
clf.get_params().keys()

dict_keys(['estimator__bootstrap', 'estimator__ccp_alpha', 'estimator__class_weight', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__max_samples', 'estimator__min_impurity_decrease', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__oob_score', 'estimator__random_state', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'n_jobs'])

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
X = train_df
y = y_label
clf = MultiOutputClassifier(
      RandomForestClassifier(n_estimators = 1700,
                             max_depth = 8,
                             #random_state = np.random.RandomState(0),
                             #min_samples_split = 7,
                             #max_features = 0.7,
                             min_samples_leaf = 5,
                             n_jobs = -1))
clf.fit(X,y)
prediction = clf.predict_proba(test_df)

In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/Colab/2212_MLSP/sample_submission.csv')
for i in range(323):
  for j in range(19):
    idx = i*19 + j
    sub.at[idx, 'probability'] = prediction[j][i][1]

sub.to_csv('/content/drive/MyDrive/Colab/2212_MLSP/RFsubmission.csv', index = False)